# L1 Language Models, the Chat Format and Tokens

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [2]:
#%pip install tiktoken 

  Using cached tiktoken-0.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached regex-2023.12.25-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
Using cached tiktoken-0.6.0-cp311-cp311-macosx_11_0_arm64.whl (949 kB)
Using cached regex-2023.12.25-cp311-cp311-macosx_11_0_arm64.whl (291 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
#%pip install load_dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached load_dotenv-0.1.0-py3-none-any.whl (7.2 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import openai
import tiktoken

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

#### helper function
This may look familiar if you took the earlier course "ChatGPT Prompt Engineering for Developers" Course. 

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat).

This helper function will make it easier to use prompts and look at the generated outputs. 

**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to produce the desired results.

In [5]:
client = openai.OpenAI()
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Prompt the model and get a completion

In [6]:
response = get_completion("What is the capital of France?")

In [7]:
print(response)

The capital of France is Paris.


## Tokens

In [8]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

pilpolol


"lollipop" in reverse should be "popillol"

In [6]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [7]:
response

'p-o-p-i-l-l-o-l'

## Helper function (chat format)
Here's the helper function we'll use in this course.

In [9]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message.content

In [10]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

Oh the happy carrot, so cheerful and bright,
Grown in the garden, a lovely sight.
With a vibrant hue and a joyful heart,
This little veggie's happiness will never depart.


In [11]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once there was a cheerful carrot named Carl who loved nothing more than basking in the warm sunshine and making friends with the garden critters.


In [12]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

In a garden so bright, there once was a carrot, with a smile so light.


In [13]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message.content
    
    token_dict = {
'prompt_tokens':response.usage.prompt_tokens, 
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens, 
    }

    return content, token_dict

In [14]:
messages = [
    {'role':'system', 
     'content':"""You are an assistant who responds\
     in the style of Dr Seuss."""},    
    {'role':'user',
     'content':"""write me a very short poem \ 
     about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)


In [15]:
print(response)

In a garden so bright, a carrot did grow,
With a smile on its face, a happy little glow.
It danced in the sun, and swayed in the breeze,
Oh, what a joyful veggie, as happy as can be!


In [16]:
print(token_dict)

{'prompt_tokens': 39, 'completion_tokens': 50, 'total_tokens': 89}


#### Notes on using the OpenAI API outside of this classroom

To install the OpenAI Python library:
```
!pip install openai
```

The library needs to be configured with your account's secret key, which is available on the [website](https://platform.openai.com/account/api-keys). 

You can either set it as the `OPENAI_API_KEY` environment variable before using the library:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

Or, set `openai.api_key` to its value:

```
import openai
openai.api_key = "sk-..."
```

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.